In [1]:
repo_url = 'https://github.com/fraimwork/fraimwork'

In [2]:
from utils.gitutils import clone_repo

# Clone the repo
repo = clone_repo(repo_url, max_size_mb=10000)
local_repo_path = str(repo.working_dir)

In [3]:
from utils.filetreeutils import FileTree

(file_tree := FileTree.from_dir(f'{local_repo_path}'))

├── .git\
│   ├── hooks\
│   │   ├── applypatch-msg.sample
│   │   ├── commit-msg.sample
│   │   ├── fsmonitor-watchman.sample
│   │   ├── post-update.sample
│   │   ├── pre-applypatch.sample
│   │   ├── pre-commit.sample
│   │   ├── pre-push.sample
│   │   ├── pre-rebase.sample
│   │   ├── pre-receive.sample
│   │   ├── prepare-commit-msg.sample
│   │   ├── update.sample
│   ├── info\
│   │   ├── exclude
│   ├── logs\
│   │   ├── refs\
│   │   │   ├── heads\
│   │   │   │   ├── main
│   │   │   ├── remotes\
│   │   │   │   ├── origin\
│   │   │   │   │   ├── HEAD
│   │   ├── HEAD
│   ├── objects\
│   │   ├── info\
│   │   ├── pack\
│   │   │   ├── pack-f7f76416c8fe0e8160a4abbe1aef076d896f4147.idx
│   │   │   ├── pack-f7f76416c8fe0e8160a4abbe1aef076d896f4147.pack
│   ├── refs\
│   │   ├── heads\
│   │   │   ├── main
│   │   ├── remotes\
│   │   │   ├── origin\
│   │   │   │   ├── HEAD
│   │   ├── tags\
│   ├── config
│   ├── description
│   ├── HEAD
│   ├── index
│   ├── packed-refs
├─

In [44]:
from utils.agent import Agent, GenerationConfig
from dotenv import load_dotenv
import os

load_dotenv()

API_KEY = os.getenv('API_KEY')

classifier = Agent(
    model_name="gemini-1.5-flash-001",
    api_key=API_KEY,
    name="classifier",
    generation_config=GenerationConfig(temperature=0.8),
    system_prompt=f"""You are a software engineer tasked with classifying the framework of a codebase. \
In the following prompts, you will be given the subdirectories of a codebase. You can respond with one of the following: \
- IDENTIFY: FRAMEWORK_NAME (if you can identify the framework based on the subdirectories)
- UNABLE (if you cannot identify the framework based on the subdirectories)"""
)

In [45]:
import re
# RegEx to match classifier responses
identify_re = re.compile(r"IDENTIFY: (.+)")
search_re = re.compile(r"OPEN: (.+)")
unable_re = re.compile(r"UNABLE")

def classify_node(node, context=[]):
    if node not in file_tree.nodes:
        print(f"Node {node} not found in file tree")
        return
    node_dict = file_tree.nodes[node]
    neighbors = list(file_tree[node])
    prompt = '\n'.join(f"├── {file_tree.nodes[k]['name']}" for k in neighbors)
    if 'content' in node_dict:
        prompt += f"\n\n{node_dict['content']}"
    response = classifier.chat(prompt)
    # Match the response to the RegEx
    if (m := identify_re.match(response)):
        print(f"{node_dict['name']} is a {m.group(1)} project")
        return m.group(1).strip()
    elif (m := search_re.match(response)):
        print(f"Opening {m.group(1)} for {node_dict['name']} project")
        to_search = f'{node}\\{m.group(1).strip()}'
        if node == '.': to_search = to_search[2:]
        return classify_node(to_search)
    elif (m := unable_re.match(response)):
        print(f"Searching {node} ...")
        return [classify_node(node) for node in neighbors if 'content' not in file_tree.nodes[node]]
    else:
        print(f"Classifier response: {response}")

(root_classification := classify_node(file_tree.root_node()))

Searching . ...
.git is a GIT  project
Searching .idx ...
Searching .vscode ...
Searching backend ...
Searching backend\app ...
Searching backend\app\utils ...
firebase is a Firebase  project
frontend is a REACT  project


['GIT', [], [], [[[]], 'Firebase'], 'REACT']